In [16]:
import json
import pandas as pd
import numpy as np
import re
from collections import Counter
from itertools import combinations

In [ ]:
# Загрузка и преобразование данных
# Файл должен быть в той же папке, что и скрипт
with open("articles_EDA.json", 'r', encoding='utf-8') as file:
    data = json.load(file)
df = pd.DataFrame([{'id': key, **value['data']} for key, value in data.items()])
print("Первые 5 строк DataFrame:")
print(df.head())
print(f"\nОбщее количество строк: {len(df)}")

Первые 5 строк DataFrame:
  id                                              title  \
0  1  Демографические итоги 2014 года. Краткий докла...   
1  2  ГРЕБНЕВАЯ МОДЕЛЬ ДЛЯ ПРОГНОЗА ДЕМОГРАФИЧЕСКИХ ...   
2  3  ЖЕНЩИНЫ И МУЖЧИНЫ: РАЗЛИЧИЯ В ПОКАЗАТЕЛЯХ РОЖД...   
3  4  Составляющие демографического потенциала сельс...   
4  5  РОССИЙСКАЯ ЭКОНОМИКА: СРЕДНЕСРОЧНЫЕ БАРЬЕРЫ И ...   

                                             authors publication_date  \
0                                         Не найдено             2015   
1  Медведева Елена Ильинична, Крошилин Сергей Вик...             2022   
2  Архангельский Владимир Николаевич, Калачикова ...             2021   
3                           Новиков В.Г., Чалый В.С.             2012   
4                           Клепач Андрей Николаевич             2021   

                                   citation_journals  \
0                          Демографическое обозрение   
1                                    Народонаселение   
2  Экономиче

In [18]:
# Анализ текстов
df['text_cleaned'] = df['text'].str.replace(r'\s+', ' ', regex=True).str.strip().fillna('')
df['text_length_chars'] = df['text'].str.len().fillna(0).astype(int)
df['text_length_words'] = df['text_cleaned'].apply(lambda x: len(re.findall(r'\b\w+\b', x)))
text_stats_chars = df['text_length_chars'].describe().round(2)
text_stats_words = df['text_length_words'].describe().round(2)
print("\nСтатистики длины текста в символах:")
print(text_stats_chars)
print("\nСтатистики длины текста в словах:")
print(text_stats_words)
missing_texts = df[df['text'].isna() | (df['text'] == '')]
short_texts = df[df['text_length_chars'] < 100]
print(f"\nКоличество отсутствующих текстов: {len(missing_texts)}")
print(f"Количество текстов короче 100 символов: {len(short_texts)}")
if not short_texts.empty:
    print("\nПримеры текстов короче 100 символов (первые 5):")
    print(short_texts[['id', 'title', 'text_length_chars', 'text']].head())
else:
    print("\nТекстов короче 100 символов не найдено.")
short_texts.to_csv('short_texts.csv', index=False, encoding='utf-8')
print("Короткие тексты сохранены в 'short_texts.csv'")


Статистики длины текста в символах:
count      5894.00
mean      29487.06
std       30710.56
min        1722.00
25%       16773.75
50%       24511.00
75%       35421.25
max      633574.00
Name: text_length_chars, dtype: float64

Статистики длины текста в словах:
count     5894.00
mean      3900.86
std       4152.81
min        236.00
25%       2155.25
50%       3170.50
75%       4664.25
max      83185.00
Name: text_length_words, dtype: float64

Количество отсутствующих текстов: 0
Количество текстов короче 100 символов: 0

Текстов короче 100 символов не найдено.
Короткие тексты сохранены в 'short_texts.csv'


In [19]:
# Анализ годов публикации
df['publication_year'] = pd.to_numeric(df['publication_date'], errors='coerce')
if df['publication_year'].dtype in ['int64', 'float64']:
    min_year = df['publication_year'].min()
    max_year = df['publication_year'].max()
    print(f"\nДиапазон годов публикации: {min_year} - {max_year}")
    year_distribution = df['publication_year'].value_counts().sort_index(ascending=False)
    print("\nРаспределение статей по годам публикации:")
    print(year_distribution)
    bins = [0, 1999, 2009, 2014, 2019, 2025]
    labels = ['Pre-2000 (1936–1999)', '2000–2009', '2010–2014', '2015–2019', '2020–2025']
    df['period'] = pd.cut(df['publication_year'], bins=bins, labels=labels, include_lowest=True)
    period_distribution = df['period'].value_counts().reindex(labels)
    period_percentages = (period_distribution / len(df) * 100).round(2)
    period_table = pd.DataFrame({
        'Publication Period': period_distribution.index,
        'Frequency': period_distribution.values,
        'Percentage (%)': period_percentages.values
    })
    period_table = pd.concat([
        period_table,
        pd.DataFrame([{'Publication Period': 'Total', 'Frequency': len(df), 'Percentage (%)': 100.0}])
    ], ignore_index=True)
    print("\nРаспределение статей по периодам публикации:")
    print(period_table.to_string(index=False))


Диапазон годов публикации: 1936 - 2025

Распределение статей по годам публикации:
publication_year
2025     48
2024    451
2023    470
2022    470
2021    450
2020    425
2019    407
2018    377
2017    313
2016    394
2015    356
2014    331
2013    267
2012    246
2011    216
2010    175
2009    104
2008    124
2007     84
2006     58
2005     42
2004     17
2003     15
2002     11
2001     11
2000      4
1999      6
1998      5
1997      4
1996      1
1992      1
1977      1
1963      1
1962      1
1960      1
1957      1
1948      1
1937      3
1936      2
Name: count, dtype: int64

Распределение статей по периодам публикации:
  Publication Period  Frequency  Percentage (%)
Pre-2000 (1936–1999)         28            0.48
           2000–2009        470            7.97
           2010–2014       1235           20.95
           2015–2019       1847           31.34
           2020–2025       2314           39.26
               Total       5894          100.00


In [23]:
# Анализ меток
def count_labels(x):
    return len(x) if isinstance(x, list) else 0
df['number_count'] = df['classification'].apply(count_labels)
label_count_distribution = df['number_count'].value_counts().sort_index()
print("\nРаспределение количества меток на статью:")
print(label_count_distribution)
avg_labels_all = df['number_count'].mean().round(2)
print(f"\nСреднее количество меток на статью (весь датасет, включая ошибки): {avg_labels_all}")
valid_df = df[df['number_count'] > 0].copy()
avg_labels_valid = valid_df['number_count'].mean().round(2)
print(f"Среднее количество меток на статью (исключая {len(df) - len(valid_df)} статей с 0 меток): {avg_labels_valid}")
print(f"Количество статей с 1+ метками: {len(valid_df)}")
all_labels = [label for labels in valid_df['classification'] for label in labels]
label_counts = Counter(all_labels)
total_labels = sum(label_counts.values())
print(f"\nОбщее количество меток во всем датасете: {total_labels}")
print("\nОбщая частота меток (целей 0–7) во всем датасете с процентами:")
for goal in range(8):
    count = label_counts.get(goal, 0)
    percentage = round((count / total_labels * 100), 2) if total_labels > 0 else 0
    print(f"Цель {goal}: {count} упоминаний ({percentage}%)")
label_freq_df = pd.DataFrame({
    'Goal': [f'Goal {i}' for i in range(8)],
    'Frequency': [label_counts.get(i, 0) for i in range(8)],
    'Percentage (%)': [round(label_counts.get(i, 0) / total_labels * 100, 2) for i in range(8)]
})
label_freq_df = pd.concat([
    label_freq_df,
    pd.DataFrame([{'Goal': 'Total', 'Frequency': total_labels, 'Percentage (%)': 100.0}])
], ignore_index=True)
print("\nТаблица частоты меток с процентами:")
print(label_freq_df.to_string(index=False))
single_label_df = valid_df[valid_df['classification'].apply(lambda x: len(x) == 1)]
single_goal_0_count = single_label_df[single_label_df['classification'].apply(lambda x: x == [0])].shape[0]
print(f"\nКоличество статей с ровно одной меткой Goal 0: {single_goal_0_count}")
multi_label_with_goal_0 = valid_df[valid_df['classification'].apply(lambda x: 0 in x and len(x) > 1)]
multi_label_with_goal_0_count = multi_label_with_goal_0.shape[0]
print(f"Количество статей, где Goal 0 встречается с другими метками: {multi_label_with_goal_0_count}")
if multi_label_with_goal_0_count > 0:
    print("\nПримеры статей, где Goal 0 встречается с другими метками (первые 5):")
    print(multi_label_with_goal_0[['id', 'title', 'classification']].head())
else:
    print("\nСтатьи с Goal 0 и другими метками не найдены.")
invalid_format_count = df[df['classification'].apply(lambda x: isinstance(x, str) and 'Неверный формат ответа' in x)].shape[0]
print(f"\nКоличество статей с 'Неверный формат ответа': {invalid_format_count}")
invalid_format_rows = df[df['classification'].apply(lambda x: isinstance(x, str) and 'Неверный формат ответа' in x)][['id', 'title', 'classification']]
if not invalid_format_rows.empty:
    print("\nПримеры строк с некорректными метками (первые 5):")
    print(invalid_format_rows.head())
else:
    print("\nСтрок с некорректными метками не найдено.")
label_pairs = [pair for labels in valid_df['classification'] if len(labels) > 1 for pair in combinations(sorted(labels), 2)]
pair_counts = Counter(label_pairs)
total_pairs = sum(pair_counts.values())
print(f"\nОбщее количество пар меток: {total_pairs}")
pair_freq_df = pd.DataFrame([
    {'Label Pair': f'[{pair[0]}, {pair[1]}]', 'Frequency': freq, 'Percentage (%)': round(freq / total_pairs * 100, 2)}
    for pair, freq in pair_counts.items()
]).sort_values(by='Frequency', ascending=False)
pair_freq_df = pd.concat([
    pair_freq_df,
    pd.DataFrame([{'Label Pair': 'Total', 'Frequency': total_pairs, 'Percentage (%)': 100.0}])
], ignore_index=True)
print("\nТаблица совместного появления меток (Label Co-occurrence):")
print(pair_freq_df.to_string(index=False))
print("\nТоп-10 парного совместного появления меток:")
print(pair_freq_df.head(10).to_string(index=False))


Распределение количества меток на статью:
number_count
0     936
1    3929
2     728
3     251
4      34
5      14
7       2
Name: count, dtype: int64

Среднее количество меток на статью (весь датасет, включая ошибки): 1.08
Среднее количество меток на статью (исключая 936 статей с 0 меток): 1.28
Количество статей с 1+ метками: 4958

Общее количество меток во всем датасете: 6358

Общая частота меток (целей 0–7) во всем датасете с процентами:
Цель 0: 541 упоминаний (8.51%)
Цель 1: 1724 упоминаний (27.12%)
Цель 2: 964 упоминаний (15.16%)
Цель 3: 915 упоминаний (14.39%)
Цель 4: 287 упоминаний (4.51%)
Цель 5: 780 упоминаний (12.27%)
Цель 6: 222 упоминаний (3.49%)
Цель 7: 925 упоминаний (14.55%)

Таблица частоты меток с процентами:
  Goal  Frequency  Percentage (%)
Goal 0        541            8.51
Goal 1       1724           27.12
Goal 2        964           15.16
Goal 3        915           14.39
Goal 4        287            4.51
Goal 5        780           12.27
Goal 6        222        

In [21]:
# Анализ ключевых слов
def extract_keywords(x):
    if isinstance(x, str):
        return [kw.strip().lower() for kw in x.split(',') if kw.strip()]
    if isinstance(x, list):
        return [str(kw).lower() for kw in x]
    return []
all_keywords = [kw for sublist in df['keywords'].apply(extract_keywords) for kw in sublist]
keyword_counts = pd.Series(Counter(all_keywords)).sort_values(ascending=False)
print("\nТоп-25 ключевых слов с количеством упоминаний (в нижнем регистре):")
print(keyword_counts.head(25))


Топ-25 ключевых слов с количеством упоминаний (в нижнем регистре):
патриотизм                           171
рождаемость                          169
россия                               157
молодежь                             146
смертность                           123
patriotism                            92
life expectancy                       90
демография                            90
устойчивое развитие                   89
регион                                89
образование                           84
патриотическое воспитание             84
здоровый образ жизни                  83
инвестиции                            83
youth                                 80
миграция                              78
бедность                              78
russia                                78
демографическая политика              76
физическая культура                   76
education                             76
mortality                             75
качество жизни                

In [22]:
# Анализ пропусков
def is_missing(val):
    if val is None:
        return True
    if isinstance(val, str):
        return val.strip() == ''
    if isinstance(val, (float, int, np.floating, np.integer, pd.Timestamp)):
        return pd.isna(val)
    return False
missing_per_column = df.apply(lambda x: x.map(is_missing)).sum().sort_values(ascending=False)
print("Пропуски по столбцам:")
print(missing_per_column)
total_missing = missing_per_column.sum()
if total_missing == 0:
    print("\nПропусков не найдено.")
else:
    print(f"\nВсего пропусков: {total_missing}")
    print("Столбцы с неполными данными:")
    print(missing_per_column[missing_per_column > 0])
cols_of_interest = ['keywords', 'anno', 'authors']
for col in cols_of_interest:
    miss_rows = df[df[col].apply(is_missing)]
    print(f"\n--- {col}: найдено {len(miss_rows)} пропуск(ов); показываю первые 10 ---")
    print(miss_rows.loc[:, ['id', 'title', col]].head(10).to_string(index=False, justify='left', max_colwidth=60))

Пропуски по столбцам:
manual_labels        5494
keywords              587
anno                  456
authors                 1
id                      0
title                   0
citation_journals       0
url                     0
text                    0
fetch_date              0
publication_date        0
ncr_category            0
classification          0
text_cleaned            0
text_length_chars       0
text_length_words       0
publication_year        0
period                  0
number_count            0
dtype: int64

Всего пропусков: 6538
Столбцы с неполными данными:
manual_labels    5494
keywords          587
anno              456
authors             1
dtype: int64

--- keywords: найдено 587 пропуск(ов); показываю первые 10 ---
id  title                                                        keywords
  6 О прогнозе долгосрочного социально-экономического развити...         
 10 Демографический и миграционный потенциалы Узбекистана Тек...         
 57 Демографическое развитие Сар